---

[//]: # "Header"

<p style="text-align: left;  font-size:18pt; LINE-HEIGHT:30px;">
    <span style="float: left">
     Technische Hochschule Ingolstadt<br>
     Prof. Dr. Sören Gröttrup
    </span>
    <span style="float: right;">
       Machine Learning 2<br>
        <span style="float: right;">SS 2025</span>
    </span>
</p>

[//]: # "Header Aufgabenblatt"
<br>
<p style="text-align: center;  font-size:18pt; LINE-HEIGHT:30px;">
     <span style="font-weight: bold;">Aufgabenblatt 3</span><br>
     Themen: Weight Initialization, Batch Normalisierung<br>
     Abgabetermin: 08.05.2025<br>
     Punkte: 25
</p>


---

**Namen:** _Bitte tragen Sie hier die Namen der Abgabegruppe ein._

---

### Daten

Der Datensatz `fashion-mnist` ist ein von Zalando bereitgestellter Datensatz und enthält Bilder von Kleidungsstücken aus 10 verschiedenen Kategorien (https://github.com/zalandoresearch/fashion-mnist).

Der Datensatz besteht aus einem Trainigsdatensatz mit 60.000 Bildern und einem Testdatensatz mit 10.000 Bildern. Jedes Bild ist ein grauskaliertes 28x28 Bild und die 10 Klassen sind mit 0 bis 9 codiert. 
Die Codierung ist in der nachfolgenden Tabelle angegeben:

|Label| Description|
| --- | --- |
|0| T-shirt/top|
|1| Trouser|
|2| Pullover|
|3| Dress|
|4| Coat|
|5| Sandal|
|6| Shirt|
|7| Sneaker|
|8| Bag|
|9| Ankle boot|

In [ ]:
# Packages
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import time

plt.style.use('ggplot')

## Aufgabe 1 [17 Punkte]

Ziel der Aufgabe ist es sich mit der Normalisierung der Input Variablen und der Initialisierung der Gewichte vertraut zu machen und dies zu implementieren.

1. Schreiben Sie einen Routine, welche das unten angegebene Neuronale Netz `Net()` mittels `F.nll_loss()` trainiert. Dabei soll während des Trainings die Accuracy des jeweiligen Batches in regelmäßigem Abstand berechnet und abgespeichert werden (alle `log_interval` Iterationen). Weiter soll nach jeder Epoche der Loss und die Accuracy des gesamten Testdatensatzes berechnet und abgespeichert werden. 

1. Trainieren Sie das Neuronale Netz mittels SGD auf dem Datensatz und visualisieren Sie die Ergebnisse. 

1. Beim Laden der Daten werden die Inputs automatisch auf das Intervall [0,1] skaliert. Fügen Sie beim Laden der Bilder den befehl `torchvision.transforms.Normalize(0.5, 0.5)` ein. Dieser skaliert die Inputs auf das Intervall [-1,1] und zentriert diese so. Trainieren Sie das Neuronale Netz erneut, visualisieren Sie die Ergebnisse und vergleichen es mit dem Training ohne zentrierte Inputs. Beschreiben Sie die Veränderungen (falls vorhanden). 

1. Informieren Sie sich darüber, wie man in PyTorch die Gewichte initialisieren kann. Fügen Sie dann dem Neuronalen Netz eine geeignete Initalisierung der Gewichte hinzu und trainieren Sie dieses dann erneut. Vergleichen Sie es mit den anderen Trainingverläufen und visualisieren Sie diese. 

1. Informieren Sie sich, wie man in PyTorch Batch-Normalisierung hinzufügen kann. Fügen Sie dem Netz an den geeigneten Stellen Batch-Normalisierung hinzu und trainieren Sie dieses erneut. Vergleichen Sie es mit den anderen Trainigsverläufen und visualisieren Sie diese. 
_Hinweis:_ Sie können das Training beschleunigen, in dem Sie im DataLoader die Option `num_workers` auf eine positive Zahl setzen (z.B. `num_workers=4`). Dies beschleunigt das Laden der einzelnen Daten für die Mini-Batches. Mehr Informationen hier: https://pytorch.org/docs/stable/data.html

## Parameter

In [ ]:
# Paths
download_data_to_folder = "../Data/"


batch_size = 64
batch_size_test = 1000

log_interval = 200
n_epochs = 10

learning_rate = 0.001

## Load data

In [ ]:
# Load train data MNIST

# Define Transformer (ToTensor() is a must)
transformer = torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor()
])

# Define data to download
train_data = torchvision.datasets.FashionMNIST(download_data_to_folder, train=True, download=True, transform=transformer)
test_data = torchvision.datasets.FashionMNIST(download_data_to_folder, train=False, download=True, transform=transformer)

In [ ]:
# Download data in one batch!
train_loader = torch.utils.data.DataLoader(
  train_data,
  batch_size=batch_size, shuffle=True)

In [ ]:
# Test Loader
test_loader = torch.utils.data.DataLoader(
  test_data,
  batch_size=batch_size_test, shuffle=False)

In [ ]:
# Initialize iterator
train_iter = iter(train_loader)
data, targets = next(train_iter)

In [ ]:
input_size = train_data[1][0].view(-1).shape[0]
print("Number of inpu variables (pixel):", input_size)

In [ ]:
# Plot images
n = 5 # number of images
start_pos = 0
for i in range(n):
  plt.subplot(1, n, i+1)
  plt.tight_layout()
  plt.imshow(data[i+start_pos][0], cmap='gray', interpolation='none')
  plt.xticks([])
  plt.yticks([])
plt.show()

## Neuronal Net

In [ ]:
# Definition of the net structure
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.input_size = input_size
        
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = x.view(-1, input_size)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim = 1)

## Beginn Ihrer Lösung

**Train and Test Funktionen**

In [ ]:
# Training function for each epoch

def train(epoch, network):
  ...

In [ ]:
# Test function that applies the test set to the trained net

def test(network):
  ...

**Run training**

In [ ]:
def run_nn(network):
    ...

## Aufgabe 2 [8 Punkte]

Trainieren Sie nun das Neuronale Netze aus Aufgabe 1 Teil 4 (mit Weight Initialization) und trainieren dies 

1. mit Momentum
1. mit Adam

jeweils mit den Default Parametern. Hat sich etwas am Trainingsverlauf geändert?


## Beginn Ihrer Lösung